In [166]:
# importing libraries

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt

In [103]:
# loading trainiter and testier

trainloader = torch.load('../../1_Data_Loader/Data_Loader_files/trainloader_V1_resnet18.pt')
testloader = torch.load('../../1_Data_Loader/Data_Loader_files/testloader_V1_resnet18.pt')

In [104]:
# testing the trainloader
train_examples = iter(trainloader)
train_samples, train_labels = train_examples.next()
print(train_samples.shape, len(train_labels))

torch.Size([16, 3, 224, 224]) 16


In [105]:
# testing the testloader
test_examples = iter(testloader)
test_samples, test_labels = test_examples.next()
print(test_samples.shape, test_labels)

torch.Size([16, 3, 224, 224]) tensor([1, 2, 0, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 2, 2, 0])


In [106]:
# setting GPU if it's available

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [107]:
# importing resnet18

model = models.resnet18(pretrained=True)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [108]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(nn.Linear(512, 10),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(10, 3),
                                 nn.LogSoftmax(dim=1))

In [111]:
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

In [112]:
# training loop



epochs = 100
steps = 0
running_loss = 0
print_every = 5
max_accuracy = 0
for epoch in range(epochs):
    for inputs, labels in trainloader:
        
        steps += 1
        # Move input and label tensors to the default device
        # inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    if accuracy >= max_accuracy:
                        max_accuracy = accuracy
                        torch.save(model.state_dict(), 'checkpoint.pth')
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()



Epoch 1/5.. Train loss: 0.567.. Test loss: 1.210.. Test accuracy: 0.547
Epoch 1/5.. Train loss: 0.566.. Test loss: 1.135.. Test accuracy: 0.552
Epoch 1/5.. Train loss: 0.514.. Test loss: 0.877.. Test accuracy: 0.581
Epoch 1/5.. Train loss: 0.413.. Test loss: 1.090.. Test accuracy: 0.555
Epoch 1/5.. Train loss: 0.457.. Test loss: 1.332.. Test accuracy: 0.574
Epoch 1/5.. Train loss: 0.480.. Test loss: 1.306.. Test accuracy: 0.571
Epoch 1/5.. Train loss: 0.459.. Test loss: 0.894.. Test accuracy: 0.585
Epoch 1/5.. Train loss: 0.445.. Test loss: 0.895.. Test accuracy: 0.572
Epoch 1/5.. Train loss: 0.460.. Test loss: 1.295.. Test accuracy: 0.536
Epoch 1/5.. Train loss: 0.631.. Test loss: 1.224.. Test accuracy: 0.592
Epoch 1/5.. Train loss: 0.462.. Test loss: 1.237.. Test accuracy: 0.567
Epoch 1/5.. Train loss: 0.667.. Test loss: 0.992.. Test accuracy: 0.535
Epoch 1/5.. Train loss: 0.506.. Test loss: 0.887.. Test accuracy: 0.541
Epoch 1/5.. Train loss: 0.396.. Test loss: 1.029.. Test accuracy

In [189]:
torch.save(model.state_dict(), "../Model_files/resnet18luca.pth")


In [190]:
import torch.onnx
from torch.autograd import Variable

dummy_input = Variable(torch.randn(1, 3, 224, 224)) 

torch.onnx.export(model,
                  dummy_input,
                  "../Model_files/resnet18_luca2.onnx",
                  verbose=True,
                    input_names=["input"],
                    output_names=["output"],
                    opset_version=11)

graph(%input : Float(1:150528, 3:50176, 224:224, 224:1, requires_grad=0, device=cpu),
      %fc.0.weight : Float(10:512, 512:1, requires_grad=1, device=cpu),
      %fc.0.bias : Float(10:1, requires_grad=1, device=cpu),
      %fc.3.weight : Float(3:10, 10:1, requires_grad=1, device=cpu),
      %fc.3.bias : Float(3:1, requires_grad=1, device=cpu),
      %198 : Float(64:147, 3:49, 7:7, 7:1, requires_grad=0, device=cpu),
      %199 : Float(64:1, requires_grad=0, device=cpu),
      %201 : Float(64:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %202 : Float(64:1, requires_grad=0, device=cpu),
      %204 : Float(64:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %205 : Float(64:1, requires_grad=0, device=cpu),
      %207 : Float(64:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %208 : Float(64:1, requires_grad=0, device=cpu),
      %210 : Float(64:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %211 : Float(64:1, requires_grad=0, device=cpu),
      %213 :

In [174]:

preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

image = 'image.jpg'
prepro_image = preprocess(image)


In [176]:
train_data_dir = '../../dataset/train/'



trainset = datasets.ImageFolder(train_data_dir, transform=preprocess)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True)


{'bad_mask': 0, 'mask': 1, 'no_mask': 2}

In [187]:
inputs, labels = next(iter(trainloader))
image = inputs[0]
pred = labels[0]

image = image.view(1, 3, 224, 224)
model(image)[0][0].dtype

torch.float32

In [188]:
image

tensor([[[[-1.7754, -1.7583, -1.7754,  ..., -1.3130, -1.2959, -1.3130],
          [-1.7925, -1.7925, -1.7754,  ..., -1.3130, -1.2959, -1.3130],
          [-1.8097, -1.8268, -1.7925,  ..., -1.2617, -1.2617, -1.2788],
          ...,
          [-1.4672, -1.5014, -1.5870,  ..., -1.7754, -1.7583, -1.7412],
          [-1.5528, -1.5014, -1.5185,  ..., -1.7925, -1.7583, -1.7412],
          [-1.6384, -1.5014, -1.4843,  ..., -1.8268, -1.7754, -1.7412]],

         [[-1.7381, -1.7031, -1.7031,  ..., -0.8627, -0.8452, -0.8277],
          [-1.7556, -1.7381, -1.7206,  ..., -0.8452, -0.8277, -0.7927],
          [-1.7556, -1.7731, -1.7381,  ..., -0.8277, -0.8102, -0.7402],
          ...,
          [-1.3529, -1.3354, -1.4580,  ..., -1.7381, -1.7206, -1.7031],
          [-1.4580, -1.3704, -1.3880,  ..., -1.7556, -1.7206, -1.7031],
          [-1.5630, -1.4055, -1.3704,  ..., -1.7906, -1.7556, -1.7031]],

         [[-1.1944, -1.2119, -1.2467,  ..., -0.2358, -0.2010, -0.2358],
          [-1.2293, -1.2467, -